## Understanding the time it takes to index images and locate the most similar image based on it features

For these experiments we will use the features of the Caltech101 dataset.
First, let's choose a random image to experiment with. We will be using the same image for all the following experiments.

In [1]:
import numpy as np
import pickle
from tqdm import notebook
import random
import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import PIL
from PIL import Image
from sklearn.neighbors import NearestNeighbors
import glob
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [2]:
filenames = pickle.load(open('data/filenames-caltech101.pickle', 'rb'))
feature_list = pickle.load(open('data/features-caltech101-resnet.pickle', 'rb'))
class_ids = pickle.load(open('data/class_ids-caltech101.pickle', 'rb'))

In [3]:
num_images = len(filenames)
num_features_per_image = len(feature_list[0])
print("Number of images: ", num_images)
print("Number of features per image: ", num_features_per_image)

Number of images:  8677
Number of features per image:  2048


In [4]:
random_image_index = random.randint(0, num_images)

# Standard features

The following experiments are based on the ResNet-50 features derived from the images of the Caltech101 dataset.

## Standard features + Brute Force Algorithm on one image

We will be timing the indexing for various Nearest Neighbors algorithms, so let's start with timing the indexing for the Brute force algorithm.

In [5]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(feature_list)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list)

2.82 ms ± 22.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Now, let's look at the time it takes to search for the nearest neighbors for the selected random image using the trained model with the Brute force algorithm.

In [6]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

11.3 ms ± 617 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Standard features + k-d Tree Algorithm on one image

Now let's turn our attention to the next nearest neighbors algoritm, the k-d tree. Let's time the indexing for the k-d tree algorithm.

In [10]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(feature_list)
neighbors = NearestNeighbors(n_neighbors=5, 
                             algorithm='kd_tree').fit(feature_list)

354 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now, time the search for the same random image using the k-d tree trained model.

In [11]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

19.1 ms ± 41.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Standard features + Ball Tree Algorithm on one image

Finally, it's time for our last nearest neighbors algorithm - the Ball Tree algorithm. As before, let's calculate the time it takes to train the model.

In [12]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(feature_list)
neighbors = NearestNeighbors(n_neighbors=5, 
                             algorithm='ball_tree').fit(feature_list)

473 ms ± 1.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


As before, let's time the search for the Ball Tree model.

In [13]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

16.9 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


We will increase the number of our test images so that we can experiment with how the scalability of different nearest neighbors algorithms change. Let's choose a random set of 100 or 1000 images to experiment.

Generate a list of images to do the next set of experiment on.

In [14]:
random_image_indices = random.sample(range(0, num_images), 1000)
random_feature_list = [
    feature_list[each_index] for each_index in random_image_indices
]

## Standard features + Brute Force Algorithm on a set of images

Time the search for the Brute force algorithm.

In [15]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

157 ms ± 1.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Standard features + k-d Tree Algorithm on a set of images

Time the search for the k-d tree algorithm.

In [16]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

18.9 s ± 28.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Standard features + Ball Tree Algorithm on a set of images

Time the search for the Ball Tree algorithm.

In [ ]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='ball_tree').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)